# 维基百科

>[维基百科](https://wikipedia.org/)是一个多语言的、由社区志愿者撰写和维护的免费在线百科全书，采用名为MediaWiki的维基编辑系统进行开放协作。 `维基百科`是历史上最大、最受欢迎的参考资料工作。

本笔记本演示了如何从`wikipedia.org`检索维基页面，以下游使用的文档格式。

## 安装

首先，您需要安装`wikipedia` python package。

In [ ]:
#!pip install wikipedia

`WikipediaRetriever`有以下参数：
- 可选的`lang`：默认为“en”。用于在维基百科的特定语言部分中搜索
- 可选的`load_max_docs`：默认=100。用它来限制下载的文档数量。下载所有100个文档需要时间，因此在实验中使用一个小数字。目前有一个硬限制为300。
- 可选的`load_all_available_meta`：默认为False。默认情况下，只下载最重要的字段：`发布时间`（发布/最后更新文档的日期）、`标题`和`摘要`。如果为True，则还会下载其他字段。

`get_relevant_documents()`有一个参数，即`query`：用于在维基百科中查找文档的自由文本

## 示例

### 运行检索器

In [28]:
from langchain.retrievers import WikipediaRetriever

In [29]:
retriever = WikipediaRetriever()

In [30]:
docs = retriever.get_relevant_documents(query='HUNTER X HUNTER')

In [31]:
docs[0].metadata  # meta-information of the Document

{'title': 'Hunter × Hunter',
 'summary': 'Hunter × Hunter (stylized as HUNTER×HUNTER and pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha\'s shōnen manga magazine Weekly Shōnen Jump since March 1998, although the manga has frequently gone on extended hiatuses since 2006. Its chapters have been collected in 37 tankōbon volumes as of November 2022. The story focuses on a young boy named Gon Freecss who discovers that his father, who left him at a young age, is actually a world-renowned Hunter, a licensed professional who specializes in fantastical pursuits such as locating rare or unidentified animal species, treasure hunting, surveying unexplored enclaves, or hunting down lawless individuals. Gon departs on a journey to become a Hunter and eventually find his father. Along the way, Gon meets various other Hunters and encounters the paranormal.\nHunter × Hunter was adapted into a 62-episode anime telev

In [32]:
docs[0].page_content[:400]  # a content of the Document 

'Hunter × Hunter (stylized as HUNTER×HUNTER and pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha\'s shōnen manga magazine Weekly Shōnen Jump since March 1998, although the manga has frequently gone on extended hiatuses since 2006. Its chapters have been collected in 37 tankōbon volumes as of November 2022. The sto'

### 基于事实的问答

In [18]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

OPENAI_API_KEY = getpass()

 ········


In [19]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name='gpt-3.5-turbo') # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model,retriever=retriever)

In [35]:
questions = [
    "What is Apify?",
    "When the Monument to the Martyrs of the 1830 Revolution was created?",
    "What is the Abhayagiri Vihāra?",   
    # "How big is Wikipédia en français?",
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What is Apify? 

**Answer**: Apify is a platform that allows you to easily automate web scraping, data extraction and web automation. It provides a cloud-based infrastructure for running web crawlers and other automation tasks, as well as a web-based tool for building and managing your crawlers. Additionally, Apify offers a marketplace for buying and selling pre-built crawlers and related services. 

-> **Question**: When the Monument to the Martyrs of the 1830 Revolution was created? 

**Answer**: Apify is a web scraping and automation platform that enables you to extract data from websites, turn unstructured data into structured data, and automate repetitive tasks. It provides a user-friendly interface for creating web scraping scripts without any coding knowledge. Apify can be used for various web scraping tasks such as data extraction, web monitoring, content aggregation, and much more. Additionally, it offers various features such as proxy support, scheduling, and